In [26]:
import numpy as np
import pandas as pd


In [27]:
videos = pd.read_csv('datasets/videos_data.csv')

In [28]:
videos.head(3)

,_id,videoFile,thumbnail,title,description,views,isPublished,categories,owner,createdAt,updatedAt
0,66e6986f0c8eb3eec2831236,https://res.cloudinary.com/dybgs03yy/video/upl...,https://res.cloudinary.com/dybgs03yy/image/upl...,Journey Through the Amazon Rainforest,"Explore the diverse ecosystems of the Amazon, ...",0,True,"nature, adventure",6661c0330f0b5138c9819cf9,2024-09-15T08:18:55.102Z,2024-09-15T08:18:55.102Z
1,66e6986f0c8eb3eec2831237,https://res.cloudinary.com/dybgs03yy/video/upl...,https://res.cloudinary.com/dybgs03yy/image/upl...,The Future of Renewable Energy,An in-depth look into the innovations in renew...,0,True,"science, environment",6661c0330f0b5138c9819cf9,2024-09-15T08:18:55.103Z,2024-09-15T08:18:55.103Z
2,66e6986f0c8eb3eec2831238,https://res.cloudinary.com/dybgs03yy/video/upl...,https://res.cloudinary.com/dybgs03yy/image/upl...,Mastering Digital Art,A comprehensive guide to digital painting tech...,0,True,"art, education",6661c0330f0b5138c9819cf9,2024-09-15T08:18:55.103Z,2024-09-15T08:18:55.103Z


In [30]:
def combine_and_split(row):
    combined_text = f"{row['title']} {row['description']} {row['categories']}"
    return combined_text.split()  


videos['tags'] = videos.apply(combine_and_split, axis=1)



In [31]:
videos['tags']

0     [Journey, Through, the, Amazon, Rainforest, Ex...
1     [The, Future, of, Renewable, Energy, An, in-de...
2     [Mastering, Digital, Art, A, comprehensive, gu...
3     [Secrets, of, Ancient, Egypt, Discover, the, m...
4     [A, Culinary, Journey, Through, Italy, Experie...
                            ...                        
94    [The, Power, of, Mindfulness:, Techniques, for...
95    [Exploring, the, Universe:, The, Role, of, Spa...
96    [The, Science, of, Nutrition:, Understanding, ...
97    [The, Future, of, Space, Exploration:, Upcomin...
98    [The, Psychology, of, Decision, Making:, How, ...
Name: tags, Length: 99, dtype: object

In [ ]:
new = videos.drop(columns=['description','','keywords','cast','crew'])


In [ ]:
new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new.head()

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [37]:
videos['tags_str'] = videos['tags'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

In [38]:
vector = cv.fit_transform(videos['tags_str']).toarray()

In [39]:
vector.shape

(99, 764)

In [40]:
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
similarity = cosine_similarity(vector)

In [42]:
similarity

array([[1.        , 0.        , 0.        , ..., 0.        , 0.03100868,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.09544271, 0.1422776 ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.03100868],
       ...,
       [0.        , 0.09544271, 0.        , ..., 1.        , 0.01720052,
        0.17200523],
       [0.03100868, 0.1422776 , 0.        , ..., 0.01720052, 1.        ,
        0.01538462],
       [0.        , 0.        , 0.03100868, ..., 0.17200523, 0.01538462,
        1.        ]])

In [66]:
def recommend(title):

    index = videos[videos['title'] == title].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
    
    for i in distances[1:6]:
        print(videos.iloc[i[0]]['title'])


In [67]:
recommend('Adventures in the Arctic')

Exploring the Galapagos Islands
Journey Through the Amazon Rainforest
Exploring the World’s Most Dangerous Animal Encounters: What You Need to Know
Cycling Through the Alps
Exploring Volcanoes Around the World


In [61]:
def recommend(title):
    index = videos[videos['title'] == title].index[0]
    
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
    
    recommended_ids = []
    for i in distances[1:6]: 
        recommended_ids.append(videos.iloc[i[0]]['_id'])
    
    return recommended_ids


In [62]:
recommend('Introduction to JavaScript')

['66e69ca32b1eb70a6795e624',
 '66e6986f0c8eb3eec2831247',
 '66e6986f0c8eb3eec2831249',
 '66e6b5bfe1c8a7042cf5b9b0',
 '66e6989ca4873093caea8595']